**2nd strategy for E.coli 5-HTP production**

In [1]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
import pandas as pd

In [ ]:
model = read_sbml_model('data/iML1515.xml')

**BH4 synthesis pathway**

GTP cyclohydrolase I\
GTP + water <-> dihydromonapterin-triphosphate + formate + H+

In [ ]:
#design reaction
gchi = Reaction('gchi')
gchi.add_metabolites({model.metabolites.gtp_c: -1,
                      model.metabolites.h2o_c: -1,
                      model.metabolites.dhmptp_c: 1,
                      model.metabolites.for_c: 1,
                      model.metabolites.h_c: 1
                     })
print(gchi.build_reaction_string())

In [ ]:
#add reaction to the system
model.add_reactions([gchi])
model.reactions.gchi

6-pyruvate-tetrahydropterin synthase\
dihydromonapterin-triphosphate <-> triphosphate + 6-pyruvoyltetrahydropterin

In [ ]:
#add metabolites
six_pyruvoyltetrahydropterin = Metabolite(id='pthp_c', compartment='c')

In [ ]:
#design reaction
ptsp = Reaction('ptsp')
ptsp.add_metabolites({model.metabolites.dhmptp_c: -1,
                      model.metabolites.pppi_c: 1,
                      six_pyruvoyltetrahydropterin: 1,
                     })
print(ptsp.build_reaction_string())

In [ ]:
#add reaction to the system
model.add_reactions([ptsp])
model.reactions.ptsp

sepiapterin reductase\
2 H+ + 2 NADPH + 6-pyruvoyltetrahydropterin <-> 2 NADP+ + tetrahydrobiopterin (BH4)

In [ ]:
# add metabolites
tetrahydrobiopterin = Metabolite(id='thbpt_c', compartment='c')

In [ ]:
#design reaction
spr = Reaction('spr')
spr.add_metabolites({model.metabolites.h_c: -2,
                      model.metabolites.nadph_c: -2,
                      six_pyruvoyltetrahydropterin: -1,
                      model.metabolites.nadp_c: 2,
                      tetrahydrobiopterin: 1
                     })
print(spr.build_reaction_string())

In [ ]:
#add reaction to the system
model.add_reactions([spr])
model.reactions.spr

**BH4 regeneration pathway**

pterin-4alpha-carbinolamine dehydratase (PCD)\
4a-hydroxytetrahydrobiopterin (hydro-BH4) <-> dihydrobiopterin (BH2) + water 

In [ ]:
#add metabolites
dihydrobiopterin = Metabolite(id='dhbpt_c', compartment='c')
four_alpha_hydroxytetrahydrobiopterin = Metabolite(id='hthbpt_c', compartment='c')

In [ ]:
#design reaction
pcd = Reaction('pcd')
pcd.add_metabolites({four_alpha_hydroxytetrahydrobiopterin: -1,
                      dihydrobiopterin: 1,
                      model.metabolites.h2o_c: 1
                     })
print(pcd.build_reaction_string())

In [ ]:
#add reaction to the system
model.add_reactions([pcd])
model.reactions.pcd

dihydropteridine reductase (DHPR)\
dihydrobiopterin (BH2) + H+ + NADPH <-> NADP+ + tetrahydrobiopterin (BH4)

In [ ]:
#design reaction
dhpr = Reaction('dhpr')
dhpr.add_metabolites({model.metabolites.dhbpt_c: -1,
                      model.metabolites.h_c: -1,
                      model.metabolites.nadph_c: -1,
                      model.metabolites.nadp_c: 1,
                      model.metabolites.thbpt_c: 1
                     })
print(dhpr.build_reaction_string())

In [ ]:
#add reaction to the system
model.add_reactions([dhpr])
model.reactions.dhpr

**5-HTP synthesis pathway**

tryptophan hydroxylase generates 5-hydroxytryptophan\
oxygen + tryptophan (trp) + tetrahydrobiopterin (BH4) <-> 5-hydroxytryptophan (5-HTP) +  4a-hydroxytetrahydrobiopterin (hydro-BH4)

In [ ]:
#add metabolites to the system
hydroxytryptophan = Metabolite(id='htrp_c', compartment='c')

In [ ]:
#design reaction
tph = Reaction('tph')
tph.add_metabolites({tetrahydrobiopterin: -1,
                      model.metabolites.o2_c: -1,
                      model.metabolites.trp__L_c: -1,
                      four_alpha_hydroxytetrahydrobiopterin: 1,
                      hydroxytryptophan: 1
                     })
print(tph.build_reaction_string())

In [ ]:
#add reaction to the system
model.add_reactions([tph])
model.reactions.tph

**Remove 5-HTP from the system**

In [ ]:
five_hydroxytryptophan_exchange = Reaction('EX_htrp')
five_hydroxytryptophan_exchange.add_metabolites({model.metabolites.htrp_c: -1})
model.add_reaction(five_hydroxytryptophan_exchange)

**Optimize model**

In [ ]:
print(model.objective)

In [ ]:
model.optimize().objective_value

**Checking max 5-HTP production and respective growth rate if objective value is changed to the 5-HTP export reaction**

In [ ]:
with model:
    print(model.reactions.tph)
    model.objective = model.reactions.tph
    solution = model.optimize()
    max_5HTP_production = solution.objective_value
    print(max_5HTP_production)
    print(solution.fluxes.BIOMASS_Ec_iML1515_core_75p37M)

**Observing how the growth rate changes with increasing 5-HTP export flux**

In [ ]:
from cobra.flux_analysis.phenotype_phase_plane import production_envelope

In [ ]:
ppp_5HTP = production_envelope(model,
                    reactions=[model.reactions.tph],
                    objective=model.reactions.BIOMASS_Ec_iML1515_core_75p37M)

In [ ]:
ppp_5HTP

In [ ]:
ppp_5HTP.plot(x='tph', y='flux_maximum')

**Optmizing 5-HTP production and growth rate**

In [ ]:
from cobra.flux_analysis import flux_variability_analysis

In [ ]:
hydroxy_trp_flux_analysis = flux_variability_analysis(model, model.reactions.tph, fraction_of_optimum = 0.5)
print(hydroxy_trp_flux_analysis)

Keeping growth rate at 50%, we get around 50% of maximum 5-HTP flux, which is bigger than in case of B.subtilis, but smaller than 1st strategy for E.coli

**Knocking out a single gene and setting the objective value of the model for 5-HTP export flux**

Only non-zero biomass production flux knock-outs are selected

In [ ]:
for gene in model.genes:
    with model:
        gene.knock_out()
        model.objective = model.reactions.tph
        solution = model.optimize()
        if solution.fluxes.BIOMASS_Ec_iML1515_core_75p37M > 0:
            print ('5-HTP export flux:', solution.fluxes.tph, 'Gene ID:', gene.id, 'Biomass production:', solution.fluxes.BIOMASS_Ec_iML1515_core_75p37M)

No solutions displayed, no single knock-outs and optmization to maximum 5-HTP production yields positive growth rate

**Knocking out a single gene, running optimisation for max biomass production flux and observing 5-HTP export flux**

Only non-zero 5-HTP export flux knock-outs are selected

In [ ]:
knock_out_list = pd.DataFrame(columns=['Gene ID', '5-HTP export flux', 'Biomass production flux'])
for gene in model.genes:
    with model:
        gene.knock_out()
        solution = model.optimize()
        if solution.fluxes.tph > 0:
            knock_out_list = knock_out_list.append({ 'Gene ID': gene.id, '5-HTP export flux': solution.fluxes.tph, 'Biomass production flux': model.optimize().objective_value}, ignore_index=True)

In [ ]:
print(knock_out_list)

No solutions, no single knock-outs give rise to positive htp flux with maximum growth rate

**Check whether change of carbon source will influence the growth rate at maximum 5-HTP flux**

In [ ]:
#create list to store the reactions' IDs
carbon_uptake = list()

#go through all metabolites in the model that have carbon in their chem.structure
for metabolite in model.metabolites:
    if 'C' in metabolite.elements:
        #find metabolites that are external
        if metabolite.id[-1] == 'e':
            reactions = metabolite.reactions
            #pick reactions which exchange the metabolite with the external environment
            for element in reactions:
                if str(element.id[:2]) == 'EX':
                    carbon_uptake.append(element.id)

In [ ]:
medium = model.medium

with model:
    #replace glucose with each exchange reaction from the possible carbon source reactions
    for reaction in carbon_uptake:
        medium['EX_glc__D_e'] = 0
        medium[reaction] = 10
        model.medium = medium
        model.objective = model.reactions.tph
        solution = model.optimize()
        max_5HTP_production = solution.objective_value
        if solution.fluxes.BIOMASS_Ec_iML1515_core_75p37M > 0:
            print(solution.fluxes.BIOMASS_Ec_iML1515_core_75p37M)
            print(max_5HTP_production)

No solutions, changing of medium does not influence the increase of growth rate at 5-HTP maximum flux production

In [ ]:
# writing the model
write_sbml_model(model, "data/iML1515_5htp_2nd_strategy.xml")